# Chapter 6

# Section 6.3.2: Full Sample Analysis(daily data)

# VAR-BEKK Model for daily data (Section 6.3.1)

In [1]:
library(xts)
library(BEKKs)
library(MTS)
library(vars)
library(stargazer)
options(repr.plot.width = 20)

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



Attaching package: ‘MTS’


The following object is masked from ‘package:BEKKs’:

    backtest


Loading required package: MASS

Loading required package: strucchange

Loading required package: sandwich

Loading required package: urca

Loading required package: lmtest


Attaching package: ‘vars’


The following object is masked from ‘package:MTS’:

    VAR



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 




# Dataset preperation

In [2]:
fr_d <- readRDS("fr_d.Rds")
fp_d <- readRDS("fp_d.Rds")
cr_d <- readRDS("cr_d.Rds")
cp_d <- readRDS("cp_d.Rds")

In [3]:
d <- na.omit(merge(cr_d,fr_d))
colnames(d) <- c("US Cocoa","London Cocoa","EURUSD","GBPUSD","CHFUSD","USDGBP","EURGBP","CHFGBP","GHSUSD","GHSGBP")
str(d)

An xts object on 2007-07-06 / 2024-05-03 containing: 
  Data:    double [4175, 10]
  Columns: US Cocoa, London Cocoa, EURUSD, GBPUSD, CHFUSD ... with 5 more columns
  Index:   Date [4175] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:216] 37 42 100 101 122 123 124 128 142 162 ...
  ..- attr(*, "index")= num [1:216] 1.19e+09 1.19e+09 1.20e+09 1.20e+09 1.20e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


In [4]:
d_p <- na.omit(merge(cp_d,fp_d))
colnames(d_p) <- c("US Cocoa","London Cocoa","EURUSD","GBPUSD","CHFUSD","USDGBP","EURGBP","CHFGBP","GHSUSD","GHSGBP")
str(d_p)

An xts object on 2007-07-05 / 2024-05-03 containing: 
  Data:    double [4176, 10]
  Columns: US Cocoa, London Cocoa, EURUSD, GBPUSD, CHFUSD ... with 5 more columns
  Index:   Date [4176] (TZ: "UTC")
  xts Attributes:
    $ na.action: 'omit' int [1:216] 38 43 101 102 123 124 125 129 143 163 ...
  ..- attr(*, "index")= num [1:216] 1.19e+09 1.19e+09 1.20e+09 1.20e+09 1.20e+09 ...


# VAR-BEKK model estimation

#### The estimation is conducted in two stages. Since the VAR model is used as the mean equation, it consitutes the first stage of the estimation process. The variance specification is the BEKK model, therefore, it is used as the second stage of the model. This procedure is followed in each case.

# (i) Spillover between US Cocoa and London Cocoa

## VAR

In [5]:
VARselect(cr_d)

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.795536e+01,-1.795359e+01,-1.795223e+01,-1.795198e+01,-1.795099e+01,-1.795014e+01,-1.795026e+01,-1.794890e+01,-1.794835e+01,-1.794712e+01
HQ(n),-1.795213e+01,-1.794821e+01,-1.794470e+01,-1.794229e+01,-1.793915e+01,-1.793615e+01,-1.793412e+01,-1.793061e+01,-1.792790e+01,-1.792452e+01
SC(n),-1.794624e+01,-1.793838e+01,-1.793094e+01,-1.792460e+01,-1.791753e+01,-1.791059e+01,-1.790463e+01,-1.789719e+01,-1.789055e+01,-1.788324e+01
FPE(n),1.592521e-08,1.595341e-08,1.597520e-08,1.597922e-08,1.599505e-08,1.600866e-08,1.600669e-08,1.602844e-08,1.603732e-08,1.605707e-08


In [6]:
var_cr<-VAR(cr_d,p = 1,type = "const")
summary(var_cr)
res_cr <- residuals(var_cr)
res_cr <- as.data.frame(res_cr)


VAR Estimation Results:
Endogenous variables: usc, lc 
Deterministic variables: const 
Sample size: 4174 
Log Likelihood: 25634.81 
Roots of the characteristic polynomial:
0.0621 0.04954
Call:
VAR(y = cr_d, p = 1, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + lc.l1 + const 

         Estimate Std. Error t value Pr(>|t|)   
usc.l1 -0.0804047  0.0325046  -2.474  0.01341 * 
lc.l1   0.1158541  0.0377888   3.066  0.00218 **
const   0.0002905  0.0002719   1.069  0.28535   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01756 on 4171 degrees of freedom
Multiple R-Squared: 0.0023,	Adjusted R-squared: 0.001821 
F-statistic: 4.807 on 2 and 4171 DF,  p-value: 0.008215 


Estimation results for equation lc: 
lc = usc.l1 + lc.l1 + const 

         Estimate Std. Error t value Pr(>|t|)  
usc.l1 -0.0205289  0.0279348  -0.735   0.4625  
lc.l1   0.0678400  0.0324761   2.089   0.0368 *
const   0.0004146  0.0002337   1.774   0.0761

In [7]:
ind_var <- index(cr_d)[-1]
str(ind_var)

 Date[1:4174], format: "2007-07-09" "2007-07-10" "2007-07-11" "2007-07-12" "2007-07-13" ...


In [8]:
res_cr <- xts(res_cr,order.by = ind_var)
str(res_cr)
head(res_cr)

An xts object on 2007-07-09 / 2024-05-03 containing: 
  Data:    double [4174, 2]
  Columns: usc, lc
  Index:   Date [4174] (TZ: "UTC")


                    usc           lc
2007-07-09 -0.011528662 -0.013937989
2007-07-10 -0.037483442 -0.040525413
2007-07-11  0.022434337  0.020815434
2007-07-12  0.004966687  0.003239114
2007-07-13 -0.001312509 -0.005129667
2007-07-16 -0.009528845 -0.009243071

## BEKK

In [9]:
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk_fit <- bekk_fit(spec = bekk_spec,data = res_cr)

In [10]:
bekk_fit$A
bekk_fit$A_t

0.17039168,-0.001807353
0.02850488,0.213457357


11.599179,-0.1374393
1.546442,13.7217288


In [11]:
bekk_fit$G
bekk_fit$G_t

0.972220313,-0.01038412
0.009970938,0.98444285


308.478434,-3.567761
2.294206,273.067805


In [12]:
summary(var_cr)
var_cr


VAR Estimation Results:
Endogenous variables: usc, lc 
Deterministic variables: const 
Sample size: 4174 
Log Likelihood: 25634.81 
Roots of the characteristic polynomial:
0.0621 0.04954
Call:
VAR(y = cr_d, p = 1, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + lc.l1 + const 

         Estimate Std. Error t value Pr(>|t|)   
usc.l1 -0.0804047  0.0325046  -2.474  0.01341 * 
lc.l1   0.1158541  0.0377888   3.066  0.00218 **
const   0.0002905  0.0002719   1.069  0.28535   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01756 on 4171 degrees of freedom
Multiple R-Squared: 0.0023,	Adjusted R-squared: 0.001821 
F-statistic: 4.807 on 2 and 4171 DF,  p-value: 0.008215 


Estimation results for equation lc: 
lc = usc.l1 + lc.l1 + const 

         Estimate Std. Error t value Pr(>|t|)  
usc.l1 -0.0205289  0.0279348  -0.735   0.4625  
lc.l1   0.0678400  0.0324761   2.089   0.0368 *
const   0.0004146  0.0002337   1.774   0.0761


VAR Estimation Results:

Estimated coefficients for equation usc: 
Call:
usc = usc.l1 + lc.l1 + const 

       usc.l1         lc.l1         const 
-0.0804047181  0.1158541497  0.0002905429 


Estimated coefficients for equation lc: 
Call:
lc = usc.l1 + lc.l1 + const 

       usc.l1         lc.l1         const 
-0.0205288827  0.0678399616  0.0004145675 



In [13]:
summary(var_cr)$varresult$lc$coefficients

,Estimate,Std. Error,t value,Pr(>|t|)
usc.l1,-0.0205288827,0.0279348044,-0.7348855,0.46245054
lc.l1,0.0678399616,0.0324761059,2.0889192,0.03677547
const,0.0004145675,0.0002336872,1.7740272,0.07613159


In [14]:
summary(var_cr)$varresult$lc$coefficients["usc.l1","Estimate"]

[1] -0.02052888

## Table summarizing the results of the VAR-BEKK model for US Cocoa - London Cocoa 

In [15]:
bekkc <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_cr)$varresult$usc$coefficients["const","Estimate"],summary(var_cr)$varresult$lc$coefficients["const","Estimate"],
                                   summary(var_cr)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_cr)$varresult$usc$coefficients["lc.l1","Estimate"],
                                   summary(var_cr)$varresult$lc$coefficients["usc.l1","Estimate"],summary(var_cr)$varresult$lc$coefficients["lc.l1","Estimate"],
                                   bekk_fit$A[1,1],bekk_fit$A[1,2],bekk_fit$A[2,1],bekk_fit$A[2,2],bekk_fit$G[1,1],bekk_fit$G[1,2],bekk_fit$G[2,1],bekk_fit$G[2,2]),
                   Std_error = c(summary(var_cr)$varresult$usc$coefficients["const","Std. Error"],summary(var_cr)$varresult$lc$coefficients["const","Std. Error"],
                               summary(var_cr)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_cr)$varresult$usc$coefficients["lc.l1","Std. Error"],
                                 summary(var_cr)$varresult$lc$coefficients["usc.l1","Std. Error"],summary(var_cr)$varresult$lc$coefficients["lc.l1","Std. Error"],
                                 bekk_fit$A_sd[1,1],bekk_fit$A_sd[1,2],bekk_fit$A_sd[2,1],bekk_fit$A_sd[2,2],bekk_fit$G_sd[1,1],bekk_fit$G_sd[1,2],bekk_fit$G_sd[2,1],bekk_fit$G_sd[2,2]),
                   T_value = c(summary(var_cr)$varresult$usc$coefficients["const","t value"],summary(var_cr)$varresult$lc$coefficients["const","t value"],
                               summary(var_cr)$varresult$usc$coefficients["usc.l1","t value"],summary(var_cr)$varresult$usc$coefficients["lc.l1","t value"],
                               summary(var_cr)$varresult$lc$coefficients["usc.l1","t value"],summary(var_cr)$varresult$lc$coefficients["lc.l1","t value"],
                               bekk_fit$A_t[1,1],bekk_fit$A_t[1,2],bekk_fit$A_t[2,1],bekk_fit$A_t[2,2],bekk_fit$G_t[1,1],bekk_fit$G_t[1,2],
                                 bekk_fit$G_t[2,1],bekk_fit$G_t[2,2]))

bekkc$Significant_at_5p_level[abs(bekkc$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekkc$Significant_at_5p_level[abs(bekkc$T_value) > 1.96 & abs(bekkc$T_value) < 2.58] <- "Stat significant at 5%"
bekkc$Significant_at_5p_level[abs(bekkc$T_value) > 1.645 & abs(bekkc$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekkc$Significant_at_5p_level[abs(bekkc$T_value) <= 1.645] <- "Not Statistically Significant"
bekkc

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,0.0002905429,0.0002719155,1.0685045,Not Statistically Significant
mu20,0.0004145675,0.0002336872,1.7740272,Stat significant at 10%
phi11,-0.0804047181,0.0325045819,-2.4736426,Stat significant at 5%
phi12,0.1158541497,0.0377887824,3.0658344,Stat significant at 1%
phi21,-0.0205288827,0.0279348044,-0.7348855,Not Statistically Significant
phi22,0.0678399616,0.0324761059,2.0889192,Stat significant at 5%
a11,0.1703916834,0.0146899780,11.5991789,Stat significant at 1%
a12,-0.0018073527,0.0131501882,-0.1374393,Not Statistically Significant
a21,0.0285048760,0.0184325569,1.5464418,Not Statistically Significant


In [16]:
causality(var_cr,cause = "usc")$Granger
causality(var_cr,cause = "lc")$Granger


	Granger causality H0: usc do not Granger-cause lc

data:  VAR object var_cr
F-Test = 0.54006, df1 = 1, df2 = 8342, p-value = 0.4624



	Granger causality H0: lc do not Granger-cause usc

data:  VAR object var_cr
F-Test = 9.3993, df1 = 1, df2 = 8342, p-value = 0.002178


## Summary for US Cocoa-London Cocoa transmission mechanism

- Return spillover from London Cocoa to New York Cocoa(lag of London cocoa is significant for US cocoa)
- no shock spillover
- volatility spillover significant in both directions

## (ii) VAR-BEKK Models for US Cocoa

In [17]:
us_d <- na.omit(merge(cr_d[,1],fr_d))
str(us_d)

An xts object on 2007-07-06 / 2024-05-03 containing: 
  Data:    double [4175, 9]
  Columns: usc, eurusd, gbpusd, chfusd, usdgbp ... with 4 more columns
  Index:   Date [4175] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:216] 37 42 100 101 122 123 124 128 142 162 ...
  ..- attr(*, "index")= num [1:216] 1.19e+09 1.19e+09 1.20e+09 1.20e+09 1.20e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


## (a) US Cocoa - EURUSD

In [18]:
bekk1data <- us_d[,c(1:2)]

In [19]:
VARselect(bekk1data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.841409e+01,-1.841235e+01,-1.841183e+01,-1.841152e+01,-1.840997e+01,-1.840982e+01,-1.840985e+01,-1.840819e+01,-1.840882e+01,-1.840902e+01
HQ(n),-1.841086e+01,-1.840697e+01,-1.840430e+01,-1.840183e+01,-1.839814e+01,-1.839583e+01,-1.839371e+01,-1.838989e+01,-1.838837e+01,-1.838642e+01
SC(n),-1.840496e+01,-1.839714e+01,-1.839054e+01,-1.838414e+01,-1.837651e+01,-1.837027e+01,-1.836422e+01,-1.835648e+01,-1.835103e+01,-1.834514e+01
FPE(n),1.006616e-08,1.008362e-08,1.008891e-08,1.009204e-08,1.010767e-08,1.010923e-08,1.010890e-08,1.012574e-08,1.011932e-08,1.011732e-08


In [20]:
var_bekk1<-VAR(bekk1data,p = 1,type = "const")
res_bekk1 <- residuals(var_bekk1)
res_bekk1 <- as.data.frame(res_bekk1)

In [21]:
summary(var_bekk1)


VAR Estimation Results:
Endogenous variables: usc, eurusd 
Deterministic variables: const 
Sample size: 4174 
Log Likelihood: 26592.686 
Roots of the characteristic polynomial:
0.04047 0.03375
Call:
VAR(y = bekk1data, p = 1, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + eurusd.l1 + const 

            Estimate Std. Error t value Pr(>|t|)    
usc.l1    -0.0067353  0.0158285  -0.426    0.670    
eurusd.l1  0.2012112  0.0475218   4.234 2.34e-05 ***
const      0.0003281  0.0002716   1.208    0.227    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01754 on 4171 degrees of freedom
Multiple R-Squared: 0.004331,	Adjusted R-squared: 0.003854 
F-statistic: 9.072 on 2 and 4171 DF,  p-value: 0.0001172 


Estimation results for equation eurusd: 
eurusd = usc.l1 + eurusd.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
usc.l1     6.788e-03  5.271e-03   1.288    0.198
eurusd.l1  1.315e-05  1.583e-02   0.001    0.

In [22]:
ind_var1 <- index(bekk1data)[-1]
str(ind_var1)

 Date[1:4174], format: "2007-07-09" "2007-07-10" "2007-07-11" "2007-07-12" "2007-07-13" ...


In [23]:
res_bekk1 <- xts(res_bekk1,order.by = ind_var1)

In [24]:
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk1_fit <- bekk_fit(spec = bekk_spec,data = res_bekk1)

In [25]:
bekk1_fit$A
bekk1_fit$A_t

0.18050952,0.0006069877
-0.05557997,0.1860307476


23.41175,0.1885821
-1.89342,20.4414776


In [26]:
bekk1_fit$G
bekk1_fit$G_t

0.9818318,2.718258e-05
0.0129997,9.805130e-01


593.502908,0.03431991
1.970149,543.33516967


- There is return spillover from EUR to US Cocoa,(significant lagged EURUSD for usc equation)
- There is a shock spillover from EUR to US Cocoa (only significant at 5% level)
- There is volatility spillover from EUR to US Cocoa significant at 5% level

### Table summarizing the results of the VAR-BEKK model for US Cocoa - EURUSD

In [27]:
bekk1 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk1)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk1)$varresult$eurusd$coefficients["const","Estimate"],
                                   summary(var_bekk1)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk1)$varresult$usc$coefficients["eurusd.l1","Estimate"],
                                   summary(var_bekk1)$varresult$eurusd$coefficients["usc.l1","Estimate"],summary(var_bekk1)$varresult$eurusd$coefficients["eurusd.l1","Estimate"],
                                   bekk1_fit$A[1,1],bekk1_fit$A[1,2],bekk1_fit$A[2,1],bekk1_fit$A[2,2],bekk1_fit$G[1,1],bekk1_fit$G[1,2],bekk1_fit$G[2,1],bekk1_fit$G[2,2]),
                   Std_error = c(summary(var_bekk1)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk1)$varresult$eurusd$coefficients["const","Std. Error"],
                                summary(var_bekk1)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk1)$varresult$usc$coefficients["eurusd.l1","Std. Error"],
                                 summary(var_bekk1)$varresult$eurusd$coefficients["usc.l1","Std. Error"],summary(var_bekk1)$varresult$eurusd$coefficients["eurusd.l1","Std. Error"],
                                 bekk1_fit$A_sd[1,1],bekk1_fit$A_sd[1,2],bekk1_fit$A_sd[2,1],bekk1_fit$A_sd[2,2],bekk1_fit$G_sd[1,1],bekk1_fit$G_sd[1,2],bekk1_fit$G_sd[2,1],bekk1_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk1)$varresult$usc$coefficients["const","t value"],summary(var_bekk1)$varresult$eurusd$coefficients["const","t value"],
                       summary(var_bekk1)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk1)$varresult$usc$coefficients["eurusd.l1","t value"],
                               summary(var_bekk1)$varresult$eurusd$coefficients["usc.l1","t value"],summary(var_bekk1)$varresult$eurusd$coefficients["eurusd.l1","t value"],
                               bekk1_fit$A_t[1,1],bekk1_fit$A_t[1,2],bekk1_fit$A_t[2,1],bekk1_fit$A_t[2,2],bekk1_fit$G_t[1,1],bekk1_fit$G_t[1,2],bekk1_fit$G_t[2,1],bekk1_fit$G_t[2,2]))
bekk1$Significant_at_5p_level[abs(bekk1$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk1$Significant_at_5p_level[abs(bekk1$T_value) > 1.96 & abs(bekk1$T_value) < 2.58] <- "Stat significant at 5%"
bekk1$Significant_at_5p_level[abs(bekk1$T_value) > 1.645 & abs(bekk1$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk1$Significant_at_5p_level[abs(bekk1$T_value) <= 1.645] <- "Not Statistically Significant"
bekk1

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,3.281255e-04,2.715539e-04,1.208326e+00,Not Statistically Significant
mu20,-5.232695e-05,9.043657e-05,-5.786039e-01,Not Statistically Significant
phi11,-6.735266e-03,1.582854e-02,-4.255142e-01,Not Statistically Significant
phi12,2.012112e-01,4.752181e-02,4.234081e+00,Stat significant at 1%
phi21,6.787967e-03,5.271435e-03,1.287689e+00,Not Statistically Significant
phi22,1.315031e-05,1.582636e-02,8.309116e-04,Not Statistically Significant
a11,1.805095e-01,7.710209e-03,2.341175e+01,Stat significant at 1%
a12,6.069877e-04,3.218691e-03,1.885821e-01,Not Statistically Significant
a21,-5.557997e-02,2.935427e-02,-1.893420e+00,Stat significant at 10%


In [28]:
causality(var_bekk1,cause = "usc")$Granger
causality(var_bekk1,cause = "eurusd")$Granger


	Granger causality H0: usc do not Granger-cause eurusd

data:  VAR object var_bekk1
F-Test = 1.6581, df1 = 1, df2 = 8342, p-value = 0.1979



	Granger causality H0: eurusd do not Granger-cause usc

data:  VAR object var_bekk1
F-Test = 17.927, df1 = 1, df2 = 8342, p-value = 2.319e-05


## (b) VAR-BEKK: US Cocoa-GBPUSD

In [29]:
bekk2data <- us_d[,c(1,3)]
head(bekk2data)

                     usc        gbpusd
2007-07-06  0.0033026687 -0.0004972527
2007-07-09 -0.0108928380  0.0022605055
2007-07-10 -0.0378502241  0.0057647029
2007-07-11  0.0210431058  0.0027838359
2007-07-12  0.0057943185 -0.0009845428
2007-07-13 -0.0009633912  0.0019435396

In [30]:
VARselect(bekk2data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.831039e+01,-1.830898e+01,-1.830797e+01,-1.830751e+01,-1.830841e+01,-1.830766e+01,-1.830892e+01,-1.830766e+01,-1.830594e+01,-1.830711e+01
HQ(n),-1.830716e+01,-1.830360e+01,-1.830044e+01,-1.829782e+01,-1.829658e+01,-1.829368e+01,-1.829278e+01,-1.828937e+01,-1.828549e+01,-1.828451e+01
SC(n),-1.830127e+01,-1.829377e+01,-1.828667e+01,-1.828013e+01,-1.827495e+01,-1.826812e+01,-1.826329e+01,-1.825595e+01,-1.824814e+01,-1.824323e+01
FPE(n),1.116602e-08,1.118183e-08,1.119311e-08,1.119827e-08,1.118812e-08,1.119651e-08,1.118250e-08,1.119651e-08,1.121585e-08,1.120271e-08


In [31]:
var_bekk2<-VAR(bekk2data,p = 1,type = "const")
res_bekk2 <- residuals(var_bekk2)
res_bekk2 <- as.data.frame(res_bekk2)

In [32]:
summary(var_bekk2)


VAR Estimation Results:
Endogenous variables: usc, gbpusd 
Deterministic variables: const 
Sample size: 4174 
Log Likelihood: 26376.746 
Roots of the characteristic polynomial:
0.0471 0.01704
Call:
VAR(y = bekk2data, p = 1, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + gbpusd.l1 + const 

            Estimate Std. Error t value Pr(>|t|)    
usc.l1    -0.0081004  0.0159291  -0.509    0.611    
gbpusd.l1  0.1848315  0.0451377   4.095  4.3e-05 ***
const      0.0003343  0.0002716   1.231    0.218    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01754 on 4171 degrees of freedom
Multiple R-Squared: 0.004055,	Adjusted R-squared: 0.003578 
F-statistic: 8.492 on 2 and 4171 DF,  p-value: 0.0002087 


Estimation results for equation gbpusd: 
gbpusd = usc.l1 + gbpusd.l1 + const 

            Estimate Std. Error t value Pr(>|t|)  
usc.l1     2.670e-03  5.616e-03   0.475   0.6345  
gbpusd.l1  3.816e-02  1.591e-02   2.398   

In [33]:
ind_var2 <- index(bekk2data)[-1]
str(ind_var2)

 Date[1:4174], format: "2007-07-09" "2007-07-10" "2007-07-11" "2007-07-12" "2007-07-13" ...


In [34]:
res_bekk2 <- xts(res_bekk2,order.by = ind_var2)

In [35]:
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk2_fit <- bekk_fit(spec = bekk_spec,data = res_bekk2)

In [36]:
bekk2_fit$A
bekk2_fit$A_t

0.178295600,-0.0006100222
0.001677049,0.2518386644


23.95432126,-0.1514288
0.06424511,34.9826249


In [37]:
bekk2_fit$G
bekk2_fit$G_t

0.982911050,0.0006984577
0.001193039,0.9606309180


600.0314183,0.6276458
0.1422559,300.3741828


- There is a significant return volatility spillover from GBP to US Cocoa
- There are no significant volatility or shock spillovers between GBP and US Cocoa

### Table summarizing the results of the VAR-BEKK model for US Cocoa - GBPUSD

In [38]:
bekk2 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk2)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk2)$varresult$gbpusd$coefficients["const","Estimate"],
                       summary(var_bekk2)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l1","Estimate"],
                                   summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l1","Estimate"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l1","Estimate"],
                                   bekk2_fit$A[1,1],bekk2_fit$A[1,2],bekk2_fit$A[2,1],bekk2_fit$A[2,2],bekk2_fit$G[1,1],bekk2_fit$G[1,2],bekk2_fit$G[2,1],bekk2_fit$G[2,2]),
                   Std_error = c(summary(var_bekk2)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk2)$varresult$gbpusd$coefficients["const","Std. Error"],
                       summary(var_bekk2)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l1","Std. Error"],
                                 summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l1","Std. Error"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l1","Std. Error"],
                                 bekk2_fit$A_sd[1,1],bekk2_fit$A_sd[1,2],bekk2_fit$A_sd[2,1],bekk2_fit$A_sd[2,2],bekk2_fit$G_sd[1,1],bekk2_fit$G_sd[1,2],bekk2_fit$G_sd[2,1],bekk2_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk2)$varresult$usc$coefficients["const","t value"],summary(var_bekk2)$varresult$gbpusd$coefficients["const","t value"],
                       summary(var_bekk2)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk2)$varresult$usc$coefficients["gbpusd.l1","t value"],
                               summary(var_bekk2)$varresult$gbpusd$coefficients["usc.l1","t value"],summary(var_bekk2)$varresult$gbpusd$coefficients["gbpusd.l1","t value"],
                               bekk2_fit$A_t[1,1],bekk2_fit$A_t[1,2],bekk2_fit$A_t[2,1],bekk2_fit$A_t[2,2],bekk2_fit$G_t[1,1],bekk2_fit$G_t[1,2],bekk2_fit$G_t[2,1],bekk2_fit$G_t[2,2]))

bekk2$Significant_at_5p_level[abs(bekk2$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk2$Significant_at_5p_level[abs(bekk2$T_value) > 1.96 & abs(bekk2$T_value) < 2.58] <- "Stat significant at 5%"
bekk2$Significant_at_5p_level[abs(bekk2$T_value) > 1.645 & abs(bekk2$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk2$Significant_at_5p_level[abs(bekk2$T_value) <= 1.645] <- "Not Statistically Significant"
bekk2

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,3.342929e-04,0.0002716165,1.23075332,Not Statistically Significant
mu20,-8.328539e-05,0.0000957667,-0.86966963,Not Statistically Significant
phi11,-8.100391e-03,0.0159290887,-0.50852822,Not Statistically Significant
phi12,1.848315e-01,0.0451376758,4.09483839,Stat significant at 1%
phi21,2.670272e-03,0.0056162875,0.47545150,Not Statistically Significant
phi22,3.815795e-02,0.0159146683,2.39765920,Stat significant at 5%
a11,1.782956e-01,0.0074431498,23.95432126,Stat significant at 1%
a12,-6.100222e-04,0.0040284411,-0.15142885,Not Statistically Significant
a21,1.677049e-03,0.0261039156,0.06424511,Not Statistically Significant


In [39]:
causality(var_bekk2,cause = "usc")$Granger
causality(var_bekk2,cause = "gbpusd")$Granger


	Granger causality H0: usc do not Granger-cause gbpusd

data:  VAR object var_bekk2
F-Test = 0.22605, df1 = 1, df2 = 8342, p-value = 0.6345



	Granger causality H0: gbpusd do not Granger-cause usc

data:  VAR object var_bekk2
F-Test = 16.768, df1 = 1, df2 = 8342, p-value = 4.265e-05


## (c) VAR-BEKK: US Cocoa-CHFUSD

In [40]:
bekk3data <- us_d[,c(1,4)]
head(bekk3data)

                     usc        chfusd
2007-07-06  0.0033026687 -0.0003044418
2007-07-09 -0.0108928380  0.0012172115
2007-07-10 -0.0378502241  0.0115503355
2007-07-11  0.0210431058 -0.0029504763
2007-07-12  0.0057943185  0.0028903495
2007-07-13 -0.0009633912  0.0000000000

In [41]:
VARselect(bekk3data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.811350e+01,-1.811192e+01,-1.811064e+01,-1.810946e+01,-1.810831e+01,-1.810775e+01,-1.810728e+01,-1.810557e+01,-1.810524e+01,-1.810475e+01
HQ(n),-1.811027e+01,-1.810654e+01,-1.810311e+01,-1.809977e+01,-1.809647e+01,-1.809376e+01,-1.809114e+01,-1.808728e+01,-1.808479e+01,-1.808215e+01
SC(n),-1.810438e+01,-1.809671e+01,-1.808934e+01,-1.808208e+01,-1.807485e+01,-1.806820e+01,-1.806165e+01,-1.805386e+01,-1.804744e+01,-1.804087e+01
FPE(n),1.359586e-08,1.361736e-08,1.363484e-08,1.365094e-08,1.366664e-08,1.367434e-08,1.368073e-08,1.370405e-08,1.370866e-08,1.371536e-08


In [42]:
var_bekk3<-VAR(bekk3data,p = 1,type = "const")
res_bekk3 <- residuals(var_bekk3)
res_bekk3 <- as.data.frame(res_bekk3)

In [43]:
summary(var_bekk3)


VAR Estimation Results:
Endogenous variables: usc, chfusd 
Deterministic variables: const 
Sample size: 4174 
Log Likelihood: 25965.156 
Roots of the characteristic polynomial:
0.02551 0.02154
Call:
VAR(y = bekk3data, p = 1, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + chfusd.l1 + const 

           Estimate Std. Error t value Pr(>|t|)  
usc.l1    0.0037354  0.0156214   0.239   0.8110  
chfusd.l1 0.0761201  0.0410353   1.855   0.0637 .
const     0.0003084  0.0002720   1.134   0.2569  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01757 on 4171 degrees of freedom
Multiple R-Squared: 0.0008757,	Adjusted R-squared: 0.0003967 
F-statistic: 1.828 on 2 and 4171 DF,  p-value: 0.1609 


Estimation results for equation chfusd: 
chfusd = usc.l1 + chfusd.l1 + const 

           Estimate Std. Error t value Pr(>|t|)
usc.l1    7.229e-03  5.936e-03   1.218    0.223
chfusd.l1 2.365e-04  1.559e-02   0.015    0.988
const     8.

In [44]:
ind_var3 <- index(bekk3data)[-1]
str(ind_var3)

 Date[1:4174], format: "2007-07-09" "2007-07-10" "2007-07-11" "2007-07-12" "2007-07-13" ...


In [45]:
res_bekk3 <- xts(res_bekk3,order.by = ind_var3)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk3_fit <- bekk_fit(spec = bekk_spec,data = res_bekk3)

In [46]:
bekk3_fit$A
bekk3_fit$A_t

0.18987810,-0.003094715
0.01508719,0.199037808


22.6551449,-0.4793143
0.5922876,18.2834288


In [47]:
bekk3_fit$G
bekk3_fit$G_t

0.980535634,-0.0001716564
0.006056191,0.9812352429


549.313694,-0.1164837
1.278309,390.7364884


- There's significant return spillover from chf to US Cocoa(significant 1st lag of CHFUSD for usc only at 10% level)
- No shock or volatility spillover

### Table summarizing the results of the VAR-BEKK model for US Cocoa - CHFUSD

In [48]:
bekk3 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk3)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk3)$varresult$chfusd$coefficients["const","Estimate"],
                       summary(var_bekk3)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk3)$varresult$usc$coefficients["chfusd.l1","Estimate"],
                                   summary(var_bekk3)$varresult$chfusd$coefficients["usc.l1","Estimate"],summary(var_bekk3)$varresult$chfusd$coefficients["chfusd.l1","Estimate"],
                                   bekk3_fit$A[1,1],bekk3_fit$A[1,2],bekk3_fit$A[2,1],bekk3_fit$A[2,2],bekk3_fit$G[1,1],bekk3_fit$G[1,2],bekk3_fit$G[2,1],bekk3_fit$G[2,2]),
                   Std_error = c(summary(var_bekk3)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk3)$varresult$chfusd$coefficients["const","Std. Error"],
                       summary(var_bekk3)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk3)$varresult$usc$coefficients["chfusd.l1","Std. Error"],
                                 summary(var_bekk3)$varresult$chfusd$coefficients["usc.l1","Std. Error"],summary(var_bekk3)$varresult$chfusd$coefficients["chfusd.l1","Std. Error"],
                                 bekk3_fit$A_sd[1,1],bekk3_fit$A_sd[1,2],bekk3_fit$A_sd[2,1],bekk3_fit$A_sd[2,2],bekk3_fit$G_sd[1,1],bekk3_fit$G_sd[1,2],bekk3_fit$G_sd[2,1],bekk3_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk3)$varresult$usc$coefficients["const","t value"],summary(var_bekk3)$varresult$chfusd$coefficients["const","t value"],
                       summary(var_bekk3)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk3)$varresult$usc$coefficients["chfusd.l1","t value"],
                               summary(var_bekk3)$varresult$chfusd$coefficients["usc.l1","t value"],summary(var_bekk3)$varresult$chfusd$coefficients["chfusd.l1","t value"],
                               bekk3_fit$A_t[1,1],bekk3_fit$A_t[1,2],bekk3_fit$A_t[2,1],bekk3_fit$A_t[2,2],bekk3_fit$G_t[1,1],bekk3_fit$G_t[1,2],bekk3_fit$G_t[2,1],bekk3_fit$G_t[2,2]))

bekk3$Significant_at_5p_level[abs(bekk3$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk3$Significant_at_5p_level[abs(bekk3$T_value) > 1.96 & abs(bekk3$T_value) < 2.58] <- "Stat significant at 5%"
bekk3$Significant_at_5p_level[abs(bekk3$T_value) > 1.645 & abs(bekk3$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk3$Significant_at_5p_level[abs(bekk3$T_value) <= 1.645] <- "Not Statistically Significant"
bekk3

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,3.084219e-04,0.0002720190,1.13382490,Not Statistically Significant
mu20,8.344983e-05,0.0001033682,0.80730700,Not Statistically Significant
phi11,3.735370e-03,0.0156213526,0.23911951,Not Statistically Significant
phi12,7.612008e-02,0.0410353374,1.85498862,Stat significant at 10%
phi21,7.228815e-03,0.0059361677,1.21775793,Not Statistically Significant
phi22,2.364666e-04,0.0155935692,0.01516437,Not Statistically Significant
a11,1.898781e-01,0.0083812354,22.65514494,Stat significant at 1%
a12,-3.094715e-03,0.0064565471,-0.47931431,Not Statistically Significant
a21,1.508719e-02,0.0254727401,0.59228764,Not Statistically Significant


In [49]:
causality(var_bekk3,cause = "usc")$Granger
causality(var_bekk3,cause = "chfusd")$Granger


	Granger causality H0: usc do not Granger-cause chfusd

data:  VAR object var_bekk3
F-Test = 1.4829, df1 = 1, df2 = 8342, p-value = 0.2234



	Granger causality H0: chfusd do not Granger-cause usc

data:  VAR object var_bekk3
F-Test = 3.441, df1 = 1, df2 = 8342, p-value = 0.06363


## (d) VAR-BEKK model for US Cocoa - GHSUSD

In [50]:
bekk7data <- na.omit(merge(cr_d[,1],fr_d[,7]))
head(bekk7data)
tail(bekk7data)

                     usc        ghsusd
2007-07-06  0.0033026687 -0.0033264938
2007-07-09 -0.0108928380  0.0040086898
2007-07-10 -0.0378502241 -0.0002705906
2007-07-11  0.0210431058  0.0026887042
2007-07-12  0.0057943185  0.0027212691
2007-07-13 -0.0009633912 -0.0051426368

                   usc       ghsusd
2024-04-26 -0.01962863 -0.003682821
2024-04-29 -0.17075943 -0.004412793
2024-04-30  0.03865640 -0.001468434
2024-05-01 -0.11385883 -0.002189577
2024-05-02 -0.09105801 -0.003637066
2024-05-03  0.07413631  0.001449549

In [51]:
VARselect(bekk7data,type = "const",lag.max = 4)

,1,2,3,4
AIC(n),-1.725174e+01,-1.725607e+01,-1.726763e+01,-1.726666e+01
HQ(n),-1.724851e+01,-1.725070e+01,-1.726011e+01,-1.725699e+01
SC(n),-1.724262e+01,-1.724088e+01,-1.724637e+01,-1.723932e+01
FPE(n),3.218597e-08,3.204661e-08,3.167832e-08,3.170920e-08


In [52]:
var_bekk7<-VAR(bekk7data,p = 3,type = "const")
res_bekk7 <- residuals(var_bekk7)
res_bekk7 <- as.data.frame(res_bekk7)

In [53]:
summary(var_bekk7)


VAR Estimation Results:
Endogenous variables: usc, ghsusd 
Deterministic variables: const 
Sample size: 4172 
Log Likelihood: 24194.877 
Roots of the characteristic polynomial:
0.507 0.4745 0.4745 0.2631 0.2461 0.2461
Call:
VAR(y = bekk7data, p = 3, type = "const")


Estimation results for equation usc: 
usc = usc.l1 + ghsusd.l1 + usc.l2 + ghsusd.l2 + usc.l3 + ghsusd.l3 + const 

            Estimate Std. Error t value Pr(>|t|)
usc.l1     0.0065533  0.0155192   0.422    0.673
ghsusd.l1 -0.0095694  0.0267626  -0.358    0.721
usc.l2     0.0064668  0.0155687   0.415    0.678
ghsusd.l2 -0.0182435  0.0268300  -0.680    0.497
usc.l3     0.0161171  0.0156490   1.030    0.303
ghsusd.l3  0.0077689  0.0267608   0.290    0.772
const      0.0003059  0.0002738   1.117    0.264


Residual standard error: 0.01758 on 4165 degrees of freedom
Multiple R-Squared: 0.0004985,	Adjusted R-squared: -0.0009414 
F-statistic: 0.3462 on 6 and 4165 DF,  p-value: 0.9124 


Estimation results for equation ghsusd: 


In [54]:
ind_var7 <- index(bekk7data)[-c(1:3)]
str(ind_var7)

 Date[1:4172], format: "2007-07-11" "2007-07-12" "2007-07-13" "2007-07-16" "2007-07-17" ...


In [55]:
res_bekk7 <- xts(res_bekk7,order.by = ind_var7)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk7_fit <- bekk_fit(spec = bekk_spec,data = res_bekk7)

In [56]:
bekk7_fit$A
bekk7_fit$A_t

0.17192358,-0.02014972
0.01556978,0.37403262


15.917118,-5.131191
1.028552,52.069073


In [57]:
bekk7_fit$G
bekk7_fit$G_t

0.94919699,0.01091725
0.00781821,0.92672258


138.450485,3.078278
1.688526,475.572219


- There is no mean spillover betweeen GHS and US Cocoa
- However, there is shock and volatility transmission from US Cocoa to GHS


### Table summarizing the results of the VAR-BEKK model for US Cocoa - GHSUSD

In [58]:
bekk7 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi13","phi14","phi15","phi16","phi21","phi22","phi23","phi24","phi25","phi26","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk7)$varresult$usc$coefficients["const","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["const","Estimate"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l1","Estimate"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l1","Estimate"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l2","Estimate"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l2","Estimate"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l3","Estimate"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l3","Estimate"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l1","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l1","Estimate"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l2","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l2","Estimate"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l3","Estimate"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l3","Estimate"],
                                   bekk7_fit$A[1,1],bekk7_fit$A[1,2],bekk7_fit$A[2,1],bekk7_fit$A[2,2],bekk7_fit$G[1,1],bekk7_fit$G[1,2],bekk7_fit$G[2,1],bekk7_fit$G[2,2]),
                   Std_error = c(summary(var_bekk7)$varresult$usc$coefficients["const","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["const","Std. Error"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l1","Std. Error"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l1","Std. Error"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l2","Std. Error"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l2","Std. Error"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l3","Std. Error"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l3","Std. Error"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l1","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l1","Std. Error"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l2","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l2","Std. Error"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l3","Std. Error"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l3","Std. Error"],
                                   bekk7_fit$A_sd[1,1],bekk7_fit$A_sd[1,2],bekk7_fit$A_sd[2,1],bekk7_fit$A_sd[2,2],bekk7_fit$G_sd[1,1],bekk7_fit$G_sd[1,2],bekk7_fit$G_sd[2,1],bekk7_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk7)$varresult$usc$coefficients["const","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["const","t value"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l1","t value"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l1","t value"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l2","t value"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l2","t value"],
                                   summary(var_bekk7)$varresult$usc$coefficients["usc.l3","t value"],summary(var_bekk7)$varresult$usc$coefficients["ghsusd.l3","t value"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l1","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l1","t value"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l2","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l2","t value"],
                                   summary(var_bekk7)$varresult$ghsusd$coefficients["usc.l3","t value"],summary(var_bekk7)$varresult$ghsusd$coefficients["ghsusd.l3","t value"],
                                   bekk7_fit$A_t[1,1],bekk7_fit$A_t[1,2],bekk7_fit$A_t[2,1],bekk7_fit$A_t[2,2],bekk7_fit$G_t[1,1],bekk7_fit$G_t[1,2],bekk7_fit$G_t[2,1],bekk7_fit$G_t[2,2]))

bekk7$Significant_at_5p_level[abs(bekk7$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk7$Significant_at_5p_level[abs(bekk7$T_value) > 1.96 & abs(bekk7$T_value) < 2.58] <- "Stat significant at 5%"
bekk7$Significant_at_5p_level[abs(bekk7$T_value) > 1.645 & abs(bekk7$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk7$Significant_at_5p_level[abs(bekk7$T_value) <= 1.645] <- "Not Statistically Significant"
bekk7

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,0.0003059434,0.0002738255,1.1172936,Not Statistically Significant
mu20,-0.0005565867,0.0001575024,-3.5338288,Stat significant at 1%
phi11,0.0065532621,0.0155192390,0.4222670,Not Statistically Significant
phi12,-0.0095694053,0.0267626497,-0.3575657,Not Statistically Significant
phi13,0.0064667894,0.0155686586,0.4153723,Not Statistically Significant
phi14,-0.0182435042,0.0268300067,-0.6799664,Not Statistically Significant
phi15,0.0161170551,0.0156490143,1.0299086,Not Statistically Significant
phi16,0.0077689004,0.0267608395,0.2903085,Not Statistically Significant
phi21,0.0081394356,0.0089265550,0.9118227,Not Statistically Significant


In [59]:
causality(var_bekk7,cause = "usc")$Granger
causality(var_bekk7,cause = "ghsusd")$Granger


	Granger causality H0: usc do not Granger-cause ghsusd

data:  VAR object var_bekk7
F-Test = 0.38624, df1 = 3, df2 = 8330, p-value = 0.7629



	Granger causality H0: ghsusd do not Granger-cause usc

data:  VAR object var_bekk7
F-Test = 0.22013, df1 = 3, df2 = 8330, p-value = 0.8825


## (iii) VAR-BEKK Models for London Cocoa

In [60]:
l_d <- na.omit(merge(cr_d[,2],fr_d[,c(4:6)]))
str(l_d)

An xts object on 2007-07-06 / 2024-05-03 containing: 
  Data:    double [4175, 4]
  Columns: lc, usdgbp, eurgbp, chfgbp
  Index:   Date [4175] (TZ: "UTC")
  xts Attributes:
    $ na.action       : 'omit' int [1:216] 37 42 100 101 122 123 124 128 142 162 ...
  ..- attr(*, "index")= num [1:216] 1.19e+09 1.19e+09 1.20e+09 1.20e+09 1.20e+09 ...
    $ ret_type        : chr "log"
    $ coredata_content: chr "logReturn"


## (a) VAR-BEKK: London Cocoa-USDGBP

In [61]:
bekk4data <- l_d[,c(1,2)]

In [62]:
VARselect(bekk4data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.855809e+01,-1.855665e+01,-1.855564e+01,-1.855507e+01,-1.855651e+01,-1.855494e+01,-1.855533e+01,-1.855382e+01,-1.855223e+01,-1.855358e+01
HQ(n),-1.855486e+01,-1.855127e+01,-1.854810e+01,-1.854539e+01,-1.854467e+01,-1.854095e+01,-1.853919e+01,-1.853553e+01,-1.853178e+01,-1.853098e+01
SC(n),-1.854896e+01,-1.854144e+01,-1.853434e+01,-1.852770e+01,-1.852305e+01,-1.851540e+01,-1.850970e+01,-1.850211e+01,-1.849443e+01,-1.848970e+01
FPE(n),8.716168e-09,8.728675e-09,8.737556e-09,8.742469e-09,8.729928e-09,8.743614e-09,8.740248e-09,8.753442e-09,8.767381e-09,8.755525e-09


In [63]:
var_bekk4<-VAR(bekk4data,p = 1,type = "const")
res_bekk4 <- residuals(var_bekk4)
res_bekk4 <- as.data.frame(res_bekk4)

In [64]:
summary(var_bekk4)


VAR Estimation Results:
Endogenous variables: lc, usdgbp 
Deterministic variables: const 
Sample size: 4174 
Log Likelihood: 26893.265 
Roots of the characteristic polynomial:
0.04588 0.04588
Call:
VAR(y = bekk4data, p = 1, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + usdgbp.l1 + const 

           Estimate Std. Error t value Pr(>|t|)   
lc.l1     0.0460288  0.0154941   2.971  0.00299 **
usdgbp.l1 0.0527223  0.0377635   1.396  0.16275   
const     0.0004132  0.0002336   1.768  0.07707 . 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01509 on 4171 degrees of freedom
Multiple R-Squared: 0.002657,	Adjusted R-squared: 0.002178 
F-statistic: 5.555 on 2 and 4171 DF,  p-value: 0.003896 


Estimation results for equation usdgbp: 
usdgbp = lc.l1 + usdgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)  
lc.l1     -5.668e-03  6.352e-03  -0.892   0.3723  
usdgbp.l1  3.923e-02  1.548e-02   2.534   0.0113 *
cons

In [65]:
ind_var4 <- index(bekk4data)[-1]
str(ind_var4)

 Date[1:4174], format: "2007-07-09" "2007-07-10" "2007-07-11" "2007-07-12" "2007-07-13" ...


In [66]:
res_bekk4 <- xts(res_bekk4,order.by = ind_var4)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk4_fit <- bekk_fit(spec = bekk_spec,data = res_bekk4)

In [67]:
bekk4_fit$A
bekk4_fit$A_t

0.19602362,0.001161249
0.05572391,0.246703248


25.102398,0.2572914
2.252496,41.7737680


In [68]:
bekk4_fit$G
bekk4_fit$G_t

0.97810060,-0.0005130079
-0.01954752,0.9622561266


527.425755,-0.3840767
-2.752008,341.6926844


- There is no significant return spillover from USD to London Cocoa
- There both shock and volatility spillovers from USD to Lodnon Cocoa

## Table summarizing the results of the VAR-BEKK model for London Cocoa - USDGBP

In [69]:
bekk4 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk4)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk4)$varresult$usdgbp$coefficients["const","Estimate"],
                                   summary(var_bekk4)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk4)$varresult$lc$coefficients["usdgbp.l1","Estimate"],
                                   summary(var_bekk4)$varresult$usdgbp$coefficients["lc.l1","Estimate"],summary(var_bekk4)$varresult$usdgbp$coefficients["usdgbp.l1","Estimate"],
                                   bekk4_fit$A[1,1],bekk4_fit$A[1,2],bekk4_fit$A[2,1],bekk4_fit$A[2,2],bekk4_fit$G[1,1],bekk4_fit$G[1,2],bekk4_fit$G[2,1],bekk4_fit$G[2,2]),
                   Std_error = c(summary(var_bekk4)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk4)$varresult$usdgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk4)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk4)$varresult$lc$coefficients["usdgbp.l1","Std. Error"],
                                   summary(var_bekk4)$varresult$usdgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk4)$varresult$usdgbp$coefficients["usdgbp.l1","Std. Error"],
                                   bekk4_fit$A_sd[1,1],bekk4_fit$A_sd[1,2],bekk4_fit$A_sd[2,1],bekk4_fit$A_sd[2,2],bekk4_fit$G_sd[1,1],bekk4_fit$G_sd[1,2],bekk4_fit$G_sd[2,1],bekk4_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk4)$varresult$lc$coefficients["const","t value"],summary(var_bekk4)$varresult$usdgbp$coefficients["const","t value"],
                                   summary(var_bekk4)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk4)$varresult$lc$coefficients["usdgbp.l1","t value"],
                                   summary(var_bekk4)$varresult$usdgbp$coefficients["lc.l1","t value"],summary(var_bekk4)$varresult$usdgbp$coefficients["usdgbp.l1","t value"],
                                   bekk4_fit$A_t[1,1],bekk4_fit$A_t[1,2],bekk4_fit$A_t[2,1],bekk4_fit$A_t[2,2],bekk4_fit$G_t[1,1],bekk4_fit$G_t[1,2],bekk4_fit$G_t[2,1],bekk4_fit$G_t[2,2]))

bekk4$Significant_at_5p_level[abs(bekk4$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk4$Significant_at_5p_level[abs(bekk4$T_value) > 1.96 & abs(bekk4$T_value) < 2.58] <- "Stat significant at 5%"
bekk4$Significant_at_5p_level[abs(bekk4$T_value) > 1.645 & abs(bekk4$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk4$Significant_at_5p_level[abs(bekk4$T_value) <= 1.645] <- "Not Statistically Significant"
bekk4

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,4.131594e-04,2.336359e-04,1.7683900,Stat significant at 10%
mu20,8.547882e-05,9.578637e-05,0.8923902,Not Statistically Significant
phi11,4.602876e-02,1.549412e-02,2.9707252,Stat significant at 1%
phi12,5.272232e-02,3.776347e-02,1.3961195,Not Statistically Significant
phi21,-5.667781e-03,6.352298e-03,-0.8922411,Not Statistically Significant
phi22,3.923287e-02,1.548232e-02,2.5340439,Stat significant at 5%
a11,1.960236e-01,7.808960e-03,25.1023979,Stat significant at 1%
a12,1.161249e-03,4.513361e-03,0.2572914,Not Statistically Significant
a21,5.572391e-02,2.473874e-02,2.2524957,Stat significant at 5%


In [70]:
causality(var_bekk4,cause = "lc")$Granger
causality(var_bekk4,cause = "usdgbp")$Granger


	Granger causality H0: lc do not Granger-cause usdgbp

data:  VAR object var_bekk4
F-Test = 0.79609, df1 = 1, df2 = 8342, p-value = 0.3723



	Granger causality H0: usdgbp do not Granger-cause lc

data:  VAR object var_bekk4
F-Test = 1.9491, df1 = 1, df2 = 8342, p-value = 0.1627


## (b) VAR-BEKK: London Cocoa-EURGBP

In [71]:
bekk5data <- l_d[,c(1,3)]

In [72]:
VARselect(bekk5data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.888253e+01,-1.888195e+01,-1.888057e+01,-1.887965e+01,-1.887906e+01,-1.887793e+01,-1.887784e+01,-1.887623e+01,-1.887570e+01,-1.887526e+01
HQ(n),-1.887930e+01,-1.887657e+01,-1.887303e+01,-1.886997e+01,-1.886722e+01,-1.886394e+01,-1.886170e+01,-1.885794e+01,-1.885526e+01,-1.885266e+01
SC(n),-1.887340e+01,-1.886674e+01,-1.885927e+01,-1.885228e+01,-1.884560e+01,-1.883839e+01,-1.883222e+01,-1.882452e+01,-1.881791e+01,-1.881138e+01
FPE(n),6.301186e-09,6.304811e-09,6.313561e-09,6.319323e-09,6.323091e-09,6.330222e-09,6.330779e-09,6.341001e-09,6.344333e-09,6.347144e-09


In [73]:
var_bekk5<-VAR(bekk5data,p = 1,type = "const")
res_bekk5 <- residuals(var_bekk5)
res_bekk5 <- as.data.frame(res_bekk5)

In [74]:
summary(var_bekk5)


VAR Estimation Results:
Endogenous variables: lc, eurgbp 
Deterministic variables: const 
Sample size: 4174 
Log Likelihood: 27569.871 
Roots of the characteristic polynomial:
0.04552 0.03187
Call:
VAR(y = bekk5data, p = 1, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + eurgbp.l1 + const 

           Estimate Std. Error t value Pr(>|t|)   
lc.l1     0.0441001  0.0155479   2.836  0.00458 **
eurgbp.l1 0.0842070  0.0444065   1.896  0.05799 . 
const     0.0004150  0.0002336   1.777  0.07570 . 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01508 on 4171 degrees of freedom
Multiple R-Squared: 0.00305,	Adjusted R-squared: 0.002572 
F-statistic:  6.38 on 2 and 4171 DF,  p-value: 0.001711 


Estimation results for equation eurgbp: 
eurgbp = lc.l1 + eurgbp.l1 + const 

           Estimate Std. Error t value Pr(>|t|)  
lc.l1     2.069e-04  5.442e-03   0.038   0.9697  
eurgbp.l1 3.330e-02  1.554e-02   2.142   0.0322 *
const   

In [75]:
ind_var5 <- index(bekk5data)[-1]
str(ind_var5)

 Date[1:4174], format: "2007-07-09" "2007-07-10" "2007-07-11" "2007-07-12" "2007-07-13" ...


In [76]:
res_bekk5 <- xts(res_bekk5,order.by = ind_var5)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk5_fit <- bekk_fit(spec = bekk_spec,data = res_bekk5)

In [77]:
bekk5_fit$A
bekk5_fit$A_t

0.20240784,-0.005305131
0.01357797,0.216535870


25.2186252,-1.663418
0.4716455,26.274679


In [78]:
bekk5_fit$G
bekk5_fit$G_t

0.9767375433,0.001751782
-0.0003320978,0.973822251


500.197342,2.017374
-0.044957,434.055553


- There's return spillover from EUR to London Cocoa(only at 10% level)
- There is volatility spillover from Lodnon Cocoa to EUR

## Table summarizing the results of the VAR-BEKK model for London Cocoa - EURGBP

In [79]:
bekk5 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk5)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk5)$varresult$eurgbp$coefficients["const","Estimate"],
                                   summary(var_bekk5)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk5)$varresult$lc$coefficients["eurgbp.l1","Estimate"],
                                   summary(var_bekk5)$varresult$eurgbp$coefficients["lc.l1","Estimate"],summary(var_bekk5)$varresult$eurgbp$coefficients["eurgbp.l1","Estimate"],
                                   bekk5_fit$A[1,1],bekk5_fit$A[1,2],bekk5_fit$A[2,1],bekk5_fit$A[2,2],bekk5_fit$G[1,1],bekk5_fit$G[1,2],bekk5_fit$G[2,1],bekk5_fit$G[2,2]),
                   Std_error = c(summary(var_bekk5)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk5)$varresult$eurgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk5)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk5)$varresult$lc$coefficients["eurgbp.l1","Std. Error"],
                                   summary(var_bekk5)$varresult$eurgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk5)$varresult$eurgbp$coefficients["eurgbp.l1","Std. Error"],
                                   bekk5_fit$A_sd[1,1],bekk5_fit$A_sd[1,2],bekk5_fit$A_sd[2,1],bekk5_fit$A_sd[2,2],bekk5_fit$G_sd[1,1],bekk5_fit$G_sd[1,2],bekk5_fit$G_sd[2,1],bekk5_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk5)$varresult$lc$coefficients["const","t value"],summary(var_bekk5)$varresult$eurgbp$coefficients["const","t value"],
                                   summary(var_bekk5)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk5)$varresult$lc$coefficients["eurgbp.l1","t value"],
                                   summary(var_bekk5)$varresult$eurgbp$coefficients["lc.l1","t value"],summary(var_bekk5)$varresult$eurgbp$coefficients["eurgbp.l1","t value"],
                                   bekk5_fit$A_t[1,1],bekk5_fit$A_t[1,2],bekk5_fit$A_t[2,1],bekk5_fit$A_t[2,2],bekk5_fit$G_t[1,1],bekk5_fit$G_t[1,2],bekk5_fit$G_t[2,1],bekk5_fit$G_t[2,2]))

bekk5$Significant_at_5p_level[abs(bekk5$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk5$Significant_at_5p_level[abs(bekk5$T_value) > 1.96 & abs(bekk5$T_value) < 2.58] <- "Stat significant at 5%"
bekk5$Significant_at_5p_level[abs(bekk5$T_value) > 1.645 & abs(bekk5$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk5$Significant_at_5p_level[abs(bekk5$T_value) <= 1.645] <- "Not Statistically Significant"
bekk5

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,4.149824e-04,2.335737e-04,1.77666566,Stat significant at 10%
mu20,4.098632e-05,8.175751e-05,0.50131573,Not Statistically Significant
phi11,4.410009e-02,1.554792e-02,2.83639728,Stat significant at 1%
phi12,8.420705e-02,4.440654e-02,1.89627585,Stat significant at 10%
phi21,2.068801e-04,5.442220e-03,0.03801391,Not Statistically Significant
phi22,3.329521e-02,1.554356e-02,2.14205768,Stat significant at 5%
a11,2.024078e-01,8.026125e-03,25.21862524,Stat significant at 1%
a12,-5.305131e-03,3.189296e-03,-1.66341768,Stat significant at 10%
a21,1.357797e-02,2.878851e-02,0.47164551,Not Statistically Significant


In [80]:
causality(var_bekk5,cause = "lc")$Granger
causality(var_bekk5,cause = "eurgbp")$Granger


	Granger causality H0: lc do not Granger-cause eurgbp

data:  VAR object var_bekk5
F-Test = 0.0014451, df1 = 1, df2 = 8342, p-value = 0.9697



	Granger causality H0: eurgbp do not Granger-cause lc

data:  VAR object var_bekk5
F-Test = 3.5959, df1 = 1, df2 = 8342, p-value = 0.05796


## (c) VAR-BEKK: London-CHFGBP

In [81]:
bekk6data <- l_d[,c(1,4)]

In [82]:
VARselect(bekk6data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.831365e+01,-1.831181e+01,-1.831036e+01,-1.830887e+01,-1.830858e+01,-1.830748e+01,-1.830713e+01,-1.830557e+01,-1.830439e+01,-1.830513e+01
HQ(n),-1.831042e+01,-1.830643e+01,-1.830283e+01,-1.829919e+01,-1.829674e+01,-1.829349e+01,-1.829099e+01,-1.828727e+01,-1.828394e+01,-1.828253e+01
SC(n),-1.830452e+01,-1.829660e+01,-1.828907e+01,-1.828150e+01,-1.827512e+01,-1.826793e+01,-1.826150e+01,-1.825386e+01,-1.824659e+01,-1.824125e+01
FPE(n),1.112971e-08,1.115017e-08,1.116633e-08,1.118300e-08,1.118628e-08,1.119861e-08,1.120252e-08,1.122002e-08,1.123326e-08,1.122495e-08


In [83]:
var_bekk6<-VAR(bekk6data,p = 1,type = "const")
res_bekk6 <- residuals(var_bekk6)
res_bekk6 <- as.data.frame(res_bekk6)

In [84]:
summary(var_bekk6)


VAR Estimation Results:
Endogenous variables: lc, chfgbp 
Deterministic variables: const 
Sample size: 4174 
Log Likelihood: 26383.018 
Roots of the characteristic polynomial:
0.04479 0.01434
Call:
VAR(y = bekk6data, p = 1, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + chfgbp.l1 + const 

           Estimate Std. Error t value Pr(>|t|)   
lc.l1     0.0462975  0.0154925   2.988  0.00282 **
chfgbp.l1 0.0367659  0.0334500   1.099  0.27177   
const     0.0004114  0.0002337   1.760  0.07842 . 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01509 on 4171 degrees of freedom
Multiple R-Squared: 0.002479,	Adjusted R-squared: 0.002001 
F-statistic: 5.184 on 2 and 4171 DF,  p-value: 0.005643 


Estimation results for equation chfgbp: 
chfgbp = lc.l1 + chfgbp.l1 + const 

            Estimate Std. Error t value Pr(>|t|)
lc.l1     -0.0013098  0.0071751  -0.183    0.855
chfgbp.l1  0.0128339  0.0154918   0.828    0.407
const     

In [85]:
ind_var6 <- index(bekk6data)[-1]
str(ind_var6)

 Date[1:4174], format: "2007-07-09" "2007-07-10" "2007-07-11" "2007-07-12" "2007-07-13" ...


In [86]:
res_bekk6 <- xts(res_bekk6,order.by = ind_var6)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk6_fit <- bekk_fit(spec = bekk_spec,data = res_bekk6)

In [87]:
bekk6_fit$A
bekk6_fit$A_t

0.20897861,-0.009019387
0.01703354,0.213859150


25.2532705,-1.166457
0.6929718,22.310933


In [88]:
bekk6_fit$G
bekk6_fit$G_t

0.975933026,0.0009673956
0.004597904,0.9493830061


498.3891682,0.4171015
0.4494223,189.7885463


No transmission between London cocoa and chf whatsoever

## Table summarizing the results of the VAR-BEKK model for London Cocoa - CHFGBP

In [89]:
bekk6 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk6)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk6)$varresult$chfgbp$coefficients["const","Estimate"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l1","Estimate"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l1","Estimate"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l1","Estimate"],
                                   bekk6_fit$A[1,1],bekk6_fit$A[1,2],bekk6_fit$A[2,1],bekk6_fit$A[2,2],bekk6_fit$G[1,1],bekk6_fit$G[1,2],bekk6_fit$G[2,1],bekk6_fit$G[2,2]),
                   Std_error = c(summary(var_bekk6)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk6)$varresult$chfgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l1","Std. Error"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l1","Std. Error"],
                                   bekk6_fit$A_sd[1,1],bekk6_fit$A_sd[1,2],bekk6_fit$A_sd[2,1],bekk6_fit$A_sd[2,2],bekk6_fit$G_sd[1,1],bekk6_fit$G_sd[1,2],bekk6_fit$G_sd[2,1],bekk6_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk6)$varresult$lc$coefficients["const","t value"],summary(var_bekk6)$varresult$chfgbp$coefficients["const","t value"],
                                   summary(var_bekk6)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk6)$varresult$lc$coefficients["chfgbp.l1","t value"],
                                   summary(var_bekk6)$varresult$chfgbp$coefficients["lc.l1","t value"],summary(var_bekk6)$varresult$chfgbp$coefficients["chfgbp.l1","t value"],
                                   bekk6_fit$A_t[1,1],bekk6_fit$A_t[1,2],bekk6_fit$A_t[2,1],bekk6_fit$A_t[2,2],bekk6_fit$G_t[1,1],bekk6_fit$G_t[1,2],bekk6_fit$G_t[2,1],bekk6_fit$G_t[2,2]))

bekk6$Significant_at_5p_level[abs(bekk6$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk6$Significant_at_5p_level[abs(bekk6$T_value) > 1.96 & abs(bekk6$T_value) < 2.58] <- "Stat significant at 5%"
bekk6$Significant_at_5p_level[abs(bekk6$T_value) > 1.645 & abs(bekk6$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk6$Significant_at_5p_level[abs(bekk6$T_value) <= 1.645] <- "Not Statistically Significant"
bekk6

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,0.0004113931,0.0002337007,1.7603421,Stat significant at 10%
mu20,0.0001689000,0.0001082342,1.5605043,Not Statistically Significant
phi11,0.0462975250,0.0154925055,2.9883820,Stat significant at 1%
phi12,0.0367658960,0.0334499871,1.0991304,Not Statistically Significant
phi21,-0.0013097675,0.0071750746,-0.1825441,Not Statistically Significant
phi22,0.0128339367,0.0154917585,0.8284364,Not Statistically Significant
a11,0.2089786053,0.0082753085,25.2532705,Stat significant at 1%
a12,-0.0090193869,0.0077322946,-1.1664567,Not Statistically Significant
a21,0.0170335373,0.0245804179,0.6929718,Not Statistically Significant


In [90]:
causality(var_bekk6,cause = "lc")$Granger
causality(var_bekk6,cause = "chfgbp")$Granger


	Granger causality H0: lc do not Granger-cause chfgbp

data:  VAR object var_bekk6
F-Test = 0.033322, df1 = 1, df2 = 8342, p-value = 0.8552



	Granger causality H0: chfgbp do not Granger-cause lc

data:  VAR object var_bekk6
F-Test = 1.2081, df1 = 1, df2 = 8342, p-value = 0.2717


## (d) VAR-BEKK Model for London Cocoa - GHSGBP

In [91]:
bekk8data <- na.omit(merge(cr_d[,2],fr_d[,8]))

In [92]:
VARselect(bekk8data,type = "const")

,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.721693e+01,-1.721912e+01,-1.722491e+01,-1.722465e+01,-1.722365e+01,-1.722231e+01,-1.722152e+01,-1.722024e+01,-1.722006e+01,-1.721945e+01
HQ(n),-1.721370e+01,-1.721374e+01,-1.721738e+01,-1.721496e+01,-1.721182e+01,-1.720833e+01,-1.720538e+01,-1.720195e+01,-1.719962e+01,-1.719685e+01
SC(n),-1.720780e+01,-1.720392e+01,-1.720362e+01,-1.719727e+01,-1.719019e+01,-1.718277e+01,-1.717589e+01,-1.716853e+01,-1.716227e+01,-1.715557e+01
FPE(n),3.332596e-08,3.325288e-08,3.306108e-08,3.306978e-08,3.310268e-08,3.314699e-08,3.317334e-08,3.321576e-08,3.322170e-08,3.324216e-08


In [93]:
var_bekk8<-VAR(bekk8data,p = 3,type = "const")
res_bekk8 <- residuals(var_bekk8)
res_bekk8 <- as.data.frame(res_bekk8)

In [94]:
summary(var_bekk8)


VAR Estimation Results:
Endogenous variables: lc, ghsgbp 
Deterministic variables: const 
Sample size: 4172 
Log Likelihood: 24110.29 
Roots of the characteristic polynomial:
0.4628 0.4251 0.4251 0.2798 0.2419 0.2419
Call:
VAR(y = bekk8data, p = 3, type = "const")


Estimation results for equation lc: 
lc = lc.l1 + ghsgbp.l1 + lc.l2 + ghsgbp.l2 + lc.l3 + ghsgbp.l3 + const 

            Estimate Std. Error t value Pr(>|t|)   
lc.l1      0.0458099  0.0155044   2.955  0.00315 **
ghsgbp.l1  0.0104334  0.0193709   0.539  0.59019   
lc.l2      0.0064808  0.0156004   0.415  0.67785   
ghsgbp.l2 -0.0035311  0.0193735  -0.182  0.85538   
lc.l3      0.0148195  0.0156477   0.947  0.34366   
ghsgbp.l3  0.0086656  0.0193710   0.447  0.65465   
const      0.0004289  0.0002345   1.829  0.06742 . 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Residual standard error: 0.01508 on 4165 degrees of freedom
Multiple R-Squared: 0.002551,	Adjusted R-squared: 0.001114 
F-statistic: 1.77

In [95]:
ind_var8 <- index(bekk8data)[-c(1:3)]
str(ind_var8)

 Date[1:4172], format: "2007-07-11" "2007-07-12" "2007-07-13" "2007-07-16" "2007-07-17" ...


In [96]:
res_bekk8 <- xts(res_bekk8,order.by = ind_var8)
bekk_spec <- bekk_spec(model = list(type = "bekk",asymmetric = F))
bekk8_fit <- bekk_fit(spec = bekk_spec,data = res_bekk8)

In [97]:
bekk8_fit$A
bekk8_fit$A_t

0.181147400,0.005401022
-0.007051672,0.351102296


27.0454489,0.836602
-0.9777392,39.455965


In [98]:
bekk8_fit$G
bekk8_fit$G_t

0.981663525,-0.0009678215
0.003342895,0.9328089008


663.210560,-0.4341755
1.198589,328.8071681


- No shock and volatility transmission

## Table summarizing the results of the VAR-BEKK model for London Cocoa - GHSGBP

In [99]:
bekk8 <- data.frame(Parameters = c("mu10","mu20","phi11","phi12","phi13","phi14","phi15","phi16","phi21","phi22","phi23","phi24","phi25","phi26","a11","a12","a21","a22","g11","g12","g21","g22"),
                   Coefficient = c(summary(var_bekk8)$varresult$lc$coefficients["const","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["const","Estimate"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l1","Estimate"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l1","Estimate"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l2","Estimate"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l2","Estimate"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l3","Estimate"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l3","Estimate"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l1","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l1","Estimate"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l2","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l2","Estimate"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l3","Estimate"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l3","Estimate"],
                                   bekk8_fit$A[1,1],bekk8_fit$A[1,2],bekk8_fit$A[2,1],bekk8_fit$A[2,2],bekk8_fit$G[1,1],bekk8_fit$G[1,2],bekk8_fit$G[2,1],bekk8_fit$G[2,2]),
                   Std_error = c(summary(var_bekk8)$varresult$lc$coefficients["const","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["const","Std. Error"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l1","Std. Error"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l1","Std. Error"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l2","Std. Error"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l2","Std. Error"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l3","Std. Error"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l3","Std. Error"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l1","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l1","Std. Error"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l2","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l2","Std. Error"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l3","Std. Error"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l3","Std. Error"],
                                   bekk8_fit$A_sd[1,1],bekk8_fit$A_sd[1,2],bekk8_fit$A_sd[2,1],bekk8_fit$A_sd[2,2],bekk8_fit$G_sd[1,1],bekk8_fit$G_sd[1,2],bekk8_fit$G_sd[2,1],bekk8_fit$G_sd[2,2]),
                   T_value = c(summary(var_bekk8)$varresult$lc$coefficients["const","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["const","t value"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l1","t value"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l1","t value"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l2","t value"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l2","t value"],
                                   summary(var_bekk8)$varresult$lc$coefficients["lc.l3","t value"],summary(var_bekk8)$varresult$lc$coefficients["ghsgbp.l3","t value"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l1","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l1","t value"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l2","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l2","t value"],
                                   summary(var_bekk8)$varresult$ghsgbp$coefficients["lc.l3","t value"],summary(var_bekk8)$varresult$ghsgbp$coefficients["ghsgbp.l3","t value"],
                                   bekk8_fit$A_t[1,1],bekk8_fit$A_t[1,2],bekk8_fit$A_t[2,1],bekk8_fit$A_t[2,2],bekk8_fit$G_t[1,1],bekk8_fit$G_t[1,2],bekk8_fit$G_t[2,1],bekk8_fit$G_t[2,2]))

bekk8$Significant_at_5p_level[abs(bekk8$T_value) >= 2.58 ] <- "Stat significant at 1%"
bekk8$Significant_at_5p_level[abs(bekk8$T_value) > 1.96 & abs(bekk8$T_value) < 2.58] <- "Stat significant at 5%"
bekk8$Significant_at_5p_level[abs(bekk8$T_value) > 1.645 & abs(bekk8$T_value) <= 1.96 ] <- "Stat significant at 10%"
bekk8$Significant_at_5p_level[abs(bekk8$T_value) <= 1.645] <- "Not Statistically Significant"
bekk8

Parameters,Coefficient,Std_error,T_value,Significant_at_5p_level
<chr>,<dbl>,<dbl>,<dbl>,<chr>
mu10,0.0004289453,0.0002344814,1.8293364,Stat significant at 10%
mu20,-0.0004576611,0.0001868663,-2.4491373,Stat significant at 5%
phi11,0.0458099434,0.0155043935,2.9546427,Stat significant at 1%
phi12,0.0104333540,0.0193709112,0.5386094,Not Statistically Significant
phi13,0.0064808069,0.0156004062,0.4154255,Not Statistically Significant
phi14,-0.0035310804,0.0193735029,-0.1822634,Not Statistically Significant
phi15,0.0148195064,0.0156476556,0.9470752,Not Statistically Significant
phi16,0.0086655818,0.0193709579,0.4473492,Not Statistically Significant
phi21,-0.0057022121,0.0123559844,-0.4614940,Not Statistically Significant


In [100]:
causality(var_bekk8,cause = "lc")$Granger
causality(var_bekk8,cause = "ghsgbp")$Granger


	Granger causality H0: lc do not Granger-cause ghsgbp

data:  VAR object var_bekk8
F-Test = 0.54737, df1 = 3, df2 = 8330, p-value = 0.6499



	Granger causality H0: ghsgbp do not Granger-cause lc

data:  VAR object var_bekk8
F-Test = 0.19535, df1 = 3, df2 = 8330, p-value = 0.8996


# Table summarizing all models

In [101]:
bekk_c <- rep(NA,14)
for(i in 1:14){
    bekk_c[i] = paste0(round(bekkc[i,2],4)," (",round(bekkc[i,3],4),")")
}

bekk_1 <- rep(NA,14)
for(i in 1:14){
    bekk_1[i] = paste0(round(bekk1[i,2],4)," (",round(bekk1[i,3],4),")")
}

bekk_2 <- rep(NA,14)
for(i in 1:14){
    bekk_2[i] = paste0(round(bekk2[i,2],4)," (",round(bekk2[i,3],4),")")
}

bekk_3 <- rep(NA,14)
for(i in 1:14){
    bekk_3[i] = paste0(round(bekk3[i,2],4)," (",round(bekk3[i,3],4),")")
}

bekk_4 <- rep(NA,14)
for(i in 1:14){
    bekk_4[i] = paste0(round(bekk4[i,2],4)," (",round(bekk4[i,3],4),")")
}

bekk_5 <- rep(NA,14)
for(i in 1:14){
    bekk_5[i] = paste0(round(bekk5[i,2],4)," (",round(bekk5[i,3],4),")")
}

bekk_6 <- rep(NA,14)
for(i in 1:14){
    bekk_6[i] = paste0(round(bekk6[i,2],4)," (",round(bekk6[i,3],4),")")
}

bekk_7 <- rep(NA,22)
for(i in 1:22){
    bekk_7[i] = paste0(round(bekk7[i,2],4)," (",round(bekk7[i,3],4),")")
}


bekk_8 <- rep(NA,22)
for(i in 1:22){
    bekk_8[i] = paste0(round(bekk8[i,2],4)," (",round(bekk8[i,3],4),")")
}


In [102]:
parameters1 = c("mu10","mu20","phi11","phi12","phi21","phi22","a11","a12","a21","a22","g11","g12","g21","g22")
parameters2 = c("mu10","mu20","phi11","phi12","phi13","phi14","phi15","phi16","phi21","phi22","phi23","phi24","phi25","phi26","a11","a12","a21","a22","g11","g12","g21","g22")

In [103]:
daily_bekk1 <- data.frame("Parameters" = parameters1,"BEKKC" = bekk_c,"BEKK1" = bekk_1,"BEKK2" = bekk_2,"BEKK3" = bekk_3,"BEKK4" = bekk_4,"BEKK5" = bekk_5,
                         "BEKK6" = bekk_6)
daily_bekk2 <- data.frame("Parameters" = parameters2,"BEKK7" = bekk_7,"BEKK8" = bekk_8)

In [104]:
daily_bekk1

Parameters,BEKKC,BEKK1,BEKK2,BEKK3,BEKK4,BEKK5,BEKK6
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
mu10,3e-04 (3e-04),3e-04 (3e-04),3e-04 (3e-04),3e-04 (3e-04),4e-04 (2e-04),4e-04 (2e-04),4e-04 (2e-04)
mu20,4e-04 (2e-04),-1e-04 (1e-04),-1e-04 (1e-04),1e-04 (1e-04),1e-04 (1e-04),0 (1e-04),2e-04 (1e-04)
phi11,-0.0804 (0.0325),-0.0067 (0.0158),-0.0081 (0.0159),0.0037 (0.0156),0.046 (0.0155),0.0441 (0.0155),0.0463 (0.0155)
phi12,0.1159 (0.0378),0.2012 (0.0475),0.1848 (0.0451),0.0761 (0.041),0.0527 (0.0378),0.0842 (0.0444),0.0368 (0.0334)
phi21,-0.0205 (0.0279),0.0068 (0.0053),0.0027 (0.0056),0.0072 (0.0059),-0.0057 (0.0064),2e-04 (0.0054),-0.0013 (0.0072)
phi22,0.0678 (0.0325),0 (0.0158),0.0382 (0.0159),2e-04 (0.0156),0.0392 (0.0155),0.0333 (0.0155),0.0128 (0.0155)
a11,0.1704 (0.0147),0.1805 (0.0077),0.1783 (0.0074),0.1899 (0.0084),0.196 (0.0078),0.2024 (0.008),0.209 (0.0083)
a12,-0.0018 (0.0132),6e-04 (0.0032),-6e-04 (0.004),-0.0031 (0.0065),0.0012 (0.0045),-0.0053 (0.0032),-0.009 (0.0077)
a21,0.0285 (0.0184),-0.0556 (0.0294),0.0017 (0.0261),0.0151 (0.0255),0.0557 (0.0247),0.0136 (0.0288),0.017 (0.0246)


In [105]:
daily_bekk2

Parameters,BEKK7,BEKK8
<chr>,<chr>,<chr>
mu10,3e-04 (3e-04),4e-04 (2e-04)
mu20,-6e-04 (2e-04),-5e-04 (2e-04)
phi11,0.0066 (0.0155),0.0458 (0.0155)
phi12,-0.0096 (0.0268),0.0104 (0.0194)
phi13,0.0065 (0.0156),0.0065 (0.0156)
phi14,-0.0182 (0.0268),-0.0035 (0.0194)
phi15,0.0161 (0.0156),0.0148 (0.0156)
phi16,0.0078 (0.0268),0.0087 (0.0194)
phi21,0.0081 (0.0089),-0.0057 (0.0124)


# Final VAR-BEKK Table for full sample of daily returns (Table 6.6)

In [106]:
daily_bekk <- data.frame("Parameters" = c(parameters2,"Port"),
                         "BEKKC"=c(bekk_c[1:4],rep(NA,4),bekk_c[5:6],rep(NA,4),bekk_c[7:14],bekk_fit$Portmanteau.test$statistic),
                         "BEKK1"=c(bekk_1[1:4],rep(NA,4),bekk_1[5:6],rep(NA,4),bekk_1[7:14],bekk1_fit$Portmanteau.test$statistic),
                         "BEKK2"=c(bekk_2[1:4],rep(NA,4),bekk_2[5:6],rep(NA,4),bekk_2[7:14],bekk2_fit$Portmanteau.test$statistic),
                          "BEKK3"=c(bekk_3[1:4],rep(NA,4),bekk_3[5:6],rep(NA,4),bekk_3[7:14],bekk3_fit$Portmanteau.test$statistic),
                          "BEKK4"=c(bekk_4[1:4],rep(NA,4),bekk_4[5:6],rep(NA,4),bekk_4[7:14],bekk4_fit$Portmanteau.test$statistic),
                         "BEKK5"=c(bekk_5[1:4],rep(NA,4),bekk_5[5:6],rep(NA,4),bekk_5[7:14],bekk5_fit$Portmanteau.test$statistic),
                          "BEKK6"=c(bekk_6[1:4],rep(NA,4),bekk_6[5:6],rep(NA,4),bekk_6[7:14],bekk6_fit$Portmanteau.test$statistic),
                         "BEKK7" = c(bekk_7,bekk7_fit$Portmanteau.test$statistic),
                         "BEKK8" = c(bekk_8,bekk8_fit$Portmanteau.test$statistic))

In [107]:
daily_bekk

Parameters,BEKKC,BEKK1,BEKK2,BEKK3,BEKK4,BEKK5,BEKK6,BEKK7,BEKK8
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
mu10,3e-04 (3e-04),3e-04 (3e-04),3e-04 (3e-04),3e-04 (3e-04),4e-04 (2e-04),4e-04 (2e-04),4e-04 (2e-04),3e-04 (3e-04),4e-04 (2e-04)
mu20,4e-04 (2e-04),-1e-04 (1e-04),-1e-04 (1e-04),1e-04 (1e-04),1e-04 (1e-04),0 (1e-04),2e-04 (1e-04),-6e-04 (2e-04),-5e-04 (2e-04)
phi11,-0.0804 (0.0325),-0.0067 (0.0158),-0.0081 (0.0159),0.0037 (0.0156),0.046 (0.0155),0.0441 (0.0155),0.0463 (0.0155),0.0066 (0.0155),0.0458 (0.0155)
phi12,0.1159 (0.0378),0.2012 (0.0475),0.1848 (0.0451),0.0761 (0.041),0.0527 (0.0378),0.0842 (0.0444),0.0368 (0.0334),-0.0096 (0.0268),0.0104 (0.0194)
phi13,NA,NA,NA,NA,NA,NA,NA,0.0065 (0.0156),0.0065 (0.0156)
phi14,NA,NA,NA,NA,NA,NA,NA,-0.0182 (0.0268),-0.0035 (0.0194)
phi15,NA,NA,NA,NA,NA,NA,NA,0.0161 (0.0156),0.0148 (0.0156)
phi16,NA,NA,NA,NA,NA,NA,NA,0.0078 (0.0268),0.0087 (0.0194)
phi21,-0.0205 (0.0279),0.0068 (0.0053),0.0027 (0.0056),0.0072 (0.0059),-0.0057 (0.0064),2e-04 (0.0054),-0.0013 (0.0072),0.0081 (0.0089),-0.0057 (0.0124)
